# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,8.62,91,100,7.72,US,1680378076
1,1,kavaratti,10.5669,72.6420,27.42,68,5,3.20,IN,1680378391
2,2,esperance,-33.8667,121.9000,17.95,84,100,5.76,AU,1680377924
3,3,punta arenas,-53.1500,-70.9167,12.06,54,20,11.32,CL,1680377875
4,4,ushuaia,-54.8000,-68.3000,8.81,61,0,3.09,AR,1680378077


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
#city_data_df = city_data_df.dropna()

fig = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    
)









                             
# Display the map
# YOUR CODE HERE
fig

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) 
                                    & (city_data_df["Max Temp"] > 21) 
                                    & (city_data_df["Wind Speed"] < 4.5) 
                                    & (city_data_df["Cloudiness"] == 0),:].dropna()
         
                                  

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,albany,42.6001,-73.9662,22.54,33,0,0.45,US,1680378055
72,72,sao filipe,14.8961,-24.4956,26.59,57,0,1.90,CV,1680378110
97,97,encruzilhada do sul,-30.5439,-52.5219,21.07,40,0,3.71,BR,1680378400
136,136,harker heights,31.0835,-97.6597,26.80,23,0,2.57,US,1680378183
196,196,tiznit,29.5833,-9.5000,23.05,33,0,0.71,MA,1680378411
237,237,santa fe,-31.6333,-60.7000,26.12,40,0,4.02,AR,1680378415
415,415,pisco,-13.7000,-76.2167,25.03,78,0,3.60,PE,1680378157
420,420,roma,43.2128,-75.4557,21.09,29,0,2.57,US,1680378143
424,424,sabha,27.0377,14.4283,21.48,11,0,3.33,LY,1680378440
486,486,jalu,29.0331,21.5482,21.53,16,0,3.07,LY,1680378106


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey": geoapify_key,
          "limit":1
          }
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row['Lat']
    lng = row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'  # YOUR CODE HERE
    params["bias"] = f'proximity:{lng},{lat}'# YOUR CODE HERE
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params= params) # YOUR CODE HERE
    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
sao filipe - nearest hotel: Bela Vista
encruzilhada do sul - nearest hotel: Hotel São Jorge
harker heights - nearest hotel: Best Western
tiznit - nearest hotel: No hotel found
santa fe - nearest hotel: Intersur Hotel
pisco - nearest hotel: La Portada
roma - nearest hotel: Wingate by Wyndham Rome
sabha - nearest hotel: فندق الكاظم-سبها
jalu - nearest hotel: فندق جالو السياحي
bastrop - nearest hotel: No hotel found
san vicente - nearest hotel: Los Lapachos
ibiruba - nearest hotel: Eurotel
nalut - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
35,albany,US,42.6001,-73.9662,33,No hotel found
72,sao filipe,CV,14.8961,-24.4956,57,Bela Vista
97,encruzilhada do sul,BR,-30.5439,-52.5219,40,Hotel São Jorge
136,harker heights,US,31.0835,-97.6597,23,Best Western
196,tiznit,MA,29.5833,-9.5000,33,No hotel found
237,santa fe,AR,-31.6333,-60.7000,40,Intersur Hotel
415,pisco,PE,-13.7000,-76.2167,78,La Portada
420,roma,US,43.2128,-75.4557,29,Wingate by Wyndham Rome
424,sabha,LY,27.0377,14.4283,11,فندق الكاظم-سبها
486,jalu,LY,29.0331,21.5482,16,فندق جالو السياحي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    xlabel='Longitude',
    ylabel='Latitude',
    title='City Map',
    size='Humidity',
    color='City',
    hover_cols=['Hotel Name', 'Country'],
    tools=['hover', 'pan', 'wheel_zoom', 'box_zoom', 'reset']
)


# Display the map
# YOUR CODE HERE
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)